In [1]:

import tvm.relay as relay
from tvm.relay.dataflow_pattern import *

In [5]:
def quant(node):
    cast_to_int8 = relay.cast(
        relay.clip(
            relay.round(
                relay.multiply(node, relay.const(8.0))
            ), 
            a_min=-127.0, a_max=127.0
        ),
        dtype="int8"
    )
    result_node = relay.annotation.stop_fusion(cast_to_int8)
    # tmp.append(result_node)
    return result_node

def dequant(node):
    cast_to_float32 = relay.divide(
        relay.cast(node, dtype='float32'), relay.const(8.0)
    )
    return cast_to_float32

def callback(self, pre, post, node_map):
    if self.pattern.match(pre):
        if pre in self.match_node:
            # print("pat 1")
            return self.dequant(self.quant(post))
            # return self.dequant(
            #         relay.layout_transform(
            #         relay.layout_transform(
            #             self.quant(post), src_layout='NCHW16c', dst_layout='NCHW'
            #         ), src_layout='NCHW', dst_layout='NCHW16c')
            #     )
    return post

In [6]:
inp = relay.var('input')


In [10]:
a = dequant(quant(inp))
a = dequant(quant(a))

In [11]:
print(a)

free_var %input;
%0 = multiply(%input, 8f);
%1 = round(%0);
%2 = clip(%1, a_min=-127f, a_max=127f);
%3 = cast(%2, dtype="int8");
%4 = annotation.stop_fusion(%3);
%5 = cast(%4, dtype="float32");
%6 = divide(%5, 8f);
%7 = multiply(%6, 8f);
%8 = round(%7);
%9 = clip(%8, a_min=-127f, a_max=127f);
%10 = cast(%9, dtype="int8");
%11 = annotation.stop_fusion(%10);
%12 = cast(%11, dtype="float32");
divide(%12, 8f)


In [43]:
class QuantizationCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        self.inp = wildcard()
        pat_mul = is_op(op_name='multiply')(self.inp, wildcard())
        pat_round = is_op(op_name='round')(pat_mul)
        pat_clip = is_op(op_name='clip')(pat_round)
        pat_cast2i8 = is_op(op_name='cast')(pat_clip)
        pat_annot = is_op(op_name='annotation.stop_fusion')(pat_cast2i8)
        pat_cast2f32 = is_op(op_name='cast')(pat_annot)
        pat_div = is_op(op_name='divide')(pat_cast2f32, wildcard())
        pat = pat_div
        self.pattern = pat

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(4.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        result_node = relay.annotation.stop_fusion(cast_to_int8)
        # tmp.append(result_node)
        return result_node

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(4.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        inp = node_map[self.inp][0]
        if self.pattern.match(pre):
            return self.dequant(self.quant(inp))
        return pre


In [44]:
qcb = QuantizationCallback()
new_a = rewrite(qcb, a)

1
1


In [45]:
print(a)

free_var %input;
%0 = multiply(%input, 8f);
%1 = round(%0);
%2 = clip(%1, a_min=-127f, a_max=127f);
%3 = cast(%2, dtype="int8");
%4 = annotation.stop_fusion(%3);
%5 = cast(%4, dtype="float32");
%6 = divide(%5, 8f);
%7 = multiply(%6, 8f);
%8 = round(%7);
%9 = clip(%8, a_min=-127f, a_max=127f);
%10 = cast(%9, dtype="int8");
%11 = annotation.stop_fusion(%10);
%12 = cast(%11, dtype="float32");
divide(%12, 8f)


In [46]:
print(new_a)

free_var %input;
%0 = multiply(%input, 4f);
%1 = round(%0);
%2 = clip(%1, a_min=-127f, a_max=127f);
%3 = cast(%2, dtype="int8");
%4 = annotation.stop_fusion(%3);
%5 = cast(%4, dtype="float32");
%6 = divide(%5, 4f);
%7 = multiply(%6, 4f);
%8 = round(%7);
%9 = clip(%8, a_min=-127f, a_max=127f);
%10 = cast(%9, dtype="int8");
%11 = annotation.stop_fusion(%10);
%12 = cast(%11, dtype="float32");
divide(%12, 4f)
